$(\textbf{x}, t)\in\Omega\times[0,\infty)$

$$\phi\frac{\partial c}{\partial t} + \textbf{u}\cdot\nabla c = \frac{1}{Ra}\nabla\cdot(\mathsf{D}\cdot\nabla c)$$

$$\textbf{u}=-\frac{\mathsf{K}}{\mu}\cdot(\nabla p + \rho\,\textbf{e}_y)$$

In [ ]:
from lucifex.fdm import FiniteDifference, AB2, CN, ConstantSeries
from lucifex.solver import BoundaryConditions, OptionsPETSc, dS_solver
from lucifex.sim import Simulation, integrate
from lucifex.utils import CellType, Perturbation, cubic_noise

from .py.porous_convection import abstract_porous_convection, create_rectangle_domain, flux


def rayleigh_darcy_convection(
    Lx: float = 2.0,
    Ly: float = 1.0,
    Nx: int = 100,
    Ny: int = 100,
    cell: str = CellType.QUADRILATERAL,
    # physical
    Ra: float = 1e3,
    # initial perturbation
    c_eps: float = 1e-6,
    c_freq: tuple[int, int] = (8, 8),
    c_seed: tuple[int, int] = (1234, 5678),
    # time step
    dt_max: float = 0.5,
    cfl_h: str | float = "hmin",
    cfl_courant: float = 0.75,
    # time discretization
    D_adv: FiniteDifference | tuple[FiniteDifference, FiniteDifference] = (AB2, CN),
    D_diff: FiniteDifference = CN,
    # linear algebra
    psi_petsc: OptionsPETSc | None = None,
    c_petsc: OptionsPETSc | None = None,
    # secondary
    secondary: bool = False,
) -> Simulation:
    
    Omega, dOmega = create_rectangle_domain(Lx, Ly, Nx, Ny, cell)
    c_bcs = BoundaryConditions(
        ("dirichlet", dOmega['lower'], 1.0),
        ("dirichlet", dOmega['upper'], 0.0),
        ('neumann', dOmega['left', 'right'], 0.0)
    )
    c_ics = Perturbation(
        lambda x: 1 - x[1],
        cubic_noise(['neumann', 'dirichlet'], [Lx, Ly], c_freq, c_seed),
        c_eps,
        [Lx, Ly],
        )   
    
    density = lambda c: -c

    simulation = abstract_porous_convection(
        Omega, 
        dOmega, 
        Ra, 
        c_ics, 
        c_bcs, 
        density=density, 
        dt_max=dt_max, 
        cfl_h=cfl_h, 
        cfl_courant=cfl_courant,
        D_adv=D_adv, 
        D_diff=D_diff, 
        psi_petsc=psi_petsc, 
        c_petsc=c_petsc, 
        secondary=secondary,
    )

    if secondary:
        c, u, d = simulation['c', 'u', 'd']
        f = ConstantSeries(Omega, "f", shape=(2, ))
        simulation.solvers.append(
            dS_solver(f, flux, lambda x: x[1] - Ly / 2, facet_side="+")(c[0], u[0], d[0], Ra),
        )

    return simulation


### $d=2$

$$\Omega=[0, L_x]\times[0, L_y]$$

$$\phi = 1$$

$$\mathsf{K}=\phi^2\mathsf{I}$$

$$\mathsf{D}=\phi\,\mathsf{I}$$

$$\rho=-c$$

$$\mu=1$$

$$c(x,y,t=0)=1-y+\mathcal{N}(x,y)$$

$$c(x,y=L_y, t)=0$$

$$c(x,y=0, t)=1$$

$$\frac{\partial c}{\partial x}(x=0,y, t)=\frac{\partial c}{\partial x}(x=L_x,y=0, t)=0$$

$$\textbf{n}\cdot\textbf{u}=0 \Longleftrightarrow \psi=0$$



In [ ]:
Lx = 2.0
Ly = 1.0
Nx = 100
Ny = 100

Ra = 500.0

c_eps = 1e-3
c_freq = (12, 8)
c_seed = (123, 456)

D_adv = AB2
D_diff = CN

simulation = rayleigh_darcy_convection(
    Lx, 
    Ly, 
    Nx, 
    Ny, 
    'quadrilateral', 
    Ra, 
    c_eps, 
    c_freq, 
    c_seed, 
    D_adv=D_adv,
    D_diff=D_diff,
)

In [ ]:
n_stop = 50
integrate(simulation, n_stop)

In [ ]:
c = simulation['c']